# Loading Dataset

In [5]:
import numpy as np 
import pandas as pd 

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
%%time
train_transaction = pd.read_csv('train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('test_transaction.csv', index_col='TransactionID')
train_identity = pd.read_csv('train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('test_identity.csv', index_col='TransactionID')
print ("Data is loaded!")

Data is loaded!
Wall time: 1min 57s


In [6]:
train_transaction.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1]:
train_transaction.card1.value_count()

NameError: name 'train_transaction' is not defined

In [7]:
train_identity.head()

,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [8]:
# Merge Datasets on TransactionID

train_df=pd.merge(train_transaction,train_identity,how="left",on="TransactionID")
test_df=pd.merge(test_transaction,test_identity,how="left",on="TransactionID")

# Modeling

In [ ]:
from sklearn import preprocessing
import warnings
warnings.simplefilter('ignore')
from sklearn.linear_model import LogisticRegression

In [14]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV

from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [11]:
train_num_df = train_df.select_dtypes(include=['float64', 'int64'])

In [12]:
# creating target and feature variables 

y_train = train_df['isFraud']

# select feature by type
X_train = train_num_df.drop('isFraud', axis=1)

In [24]:
y_train.unique()

array([0, 1], dtype=int64)

## Create smaller dataset for investigation 

In [ ]:
# Create a smaller dataset for investigation purpose only
sample_size = 20000
tmp = data_preprocessed.sample(sample_size, random_state=414)
X_small = tmp.drop(columns=['isFraud'])
y_small = tmp['isFraud'].replace({1: 0, 4: 0, 2: 1, 3: 1})

## Simple Model (first iteration)

In [ ]:
# create Basemodel: SGDClassifier Logistic Regression 

log_reg_model = LogisticRegression(class_weight='balanced')
cross_val_score(log_reg_model, X_train, y_train, cv=3, scoring='recall')

base_model = SGDClassifier(loss='log', alpha=0.5, class_weight='balanced')
cv_results_base_model = cross_validate(base_model, X_train, y_train, cv=5, n_jobs=1, scoring=['recall', 'f1_macro'])
cv_results_base_model['test_f1_macro'].mean()

In [ ]:
# Logistic Regression Model 

log_reg_model = LogisticRegression(class_weight='balanced')
cv_results_log_reg_model = cross_val_score(log_reg_model, X_train, y_train, cv=5, scoring=['recall', 'f1_macro'])

## Advanced Models 

In [ ]:
models = []

models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('SVM', SVC()))
models.append(('XGB', XGBClassifier()))
models.append(('RF', RandomForestClassifier()))

#testing models

results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=10, random_state=42)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc')
    results.append(cv_results)
    names.append(name)
    msg = '%s: %f (%f)' % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Simple Decision Tree 

from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(class_weight='balanced')
cv_result_dt_model = cross_val_score(dt_model, X_train, y_train, cv=3, scoring=['recall', 'f1_macro'])

### Random Forest Classifier 

In [ ]:
# Simple Random Forest 

from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(class_weight='balanced', random_state=0)
cv_results_forest = cross_validate(forest, X_train, y_train, cv=5, scoring='f1_macro')
print(cv_results_forest['test_score'].mean())

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
# Do we overfit on our train set?
f1_score(forest.predict(X_train_small), y_train_small, average='macro')

In [ ]:
print(classification_report(forest.predict(X_test_small), y_test_small))

Hyperparameter Tuning 

In [ ]:
# Wide RandomSearch

model = RandomForestClassifier(class_weight='balanced')

search_space = {'n_estimators': [int(x) for x in np.linspace(50, 1000, num=20)],
                'max_depth': [int(x) for x in np.linspace(10, 100, num=10)] + [None],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4],
                'criterion': ['gini'],
                'bootstrap': [True, False]
                }

cv_model = RandomizedSearchCV(model,
                              scoring='f1_macro',
                              param_distributions=search_space,
                              n_jobs=-1,
                              cv=3,
                              n_iter=30,
                              verbose=1)

search = cv_model.fit(X_train_small, y_train_small)

In [ ]:
search.best_score_

In [ ]:
search.best_params_

In [ ]:
best_forest = RandomForestClassifier(**{'n_estimators': 300, 'max_depth': 15, 'bootstrap': False})
cross_validate(best_forest, X_train_small, y_train_small, cv=5, scoring='f1_macro')['test_score'].mean()

In [ ]:
# Final Evaluation!
print(classification_report(best_forest.fit(X_train_small, y_train_small).predict(X_test_small), y_test_small))

In [ ]:
# Improving the model: Grid Search 

from sklearn.model_selection import ParameterGrid

# Create a dictionary of hyperparameters to search
grid = {'n_estimators':[100,150,200,250], 'max_depth': [5,10,15,20], 'max_features': [10,15,20,50,100], 'random_state': [0]}
test_scores = []

# Loop through the parameter grid, set the hyperparameters, and save the scores
for g in ParameterGrid(grid):
    m.set_params(**g)  # ** is "unpacking" the dictionary
    m.fit(X_train, y_train)
    #test_scores.append(m.score(X_val, y_val))
    test_scores.append(roc_auc_score(y_val, m.predict_proba(X_val)[:,1]))

# Find best hyperparameters from the test score and print
best_idx = np.argmax(test_scores)
print(test_scores[best_idx], ParameterGrid(grid)[best_idx])

# pick the best results
m.set_params(**ParameterGrid(grid)[best_idx])